## 2.Data Analysis

The second phase of the Titanic Project is the Data analysis. On this stage the tasks to be accoplished is: download the data from the source, a description of the fields on the data, exploratory analysis and quality checkup. The objective of the phase is , with the main objective proposed on the first stage, identify, collect and analise what has to be made on the data to obtain a satisfactory result at the end.  

### A) Download the data

The data is located at the kaggle website:https://www.kaggle.com/competitions/titanic/data. Next, we will download the data into the data directory.

In [3]:
!kaggle competitions download -c titanic -p ../data/

  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|███████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 414kB/s]


Unpacking the files.

In [4]:
! unzip ../data/titanic.zip -d ../data/

Archive:  ../data/titanic.zip
  inflating: ../data/gender_submission.csv  
  inflating: ../data/test.csv        
  inflating: ../data/train.csv       


Now we can proceed loading the data.

### B) Describe the data

The data is on a CSV format and subdivided into 3 files. In this section we will load and describe the data using the pandas library.

In [6]:
import pandas as pd
#loading the data
gender_submission = pd.read_csv("../data/gender_submission.csv")
test_ds = pd.read_csv("../data/test.csv")
train_ds = pd.read_csv("../data/train.csv")

On the website of the competition, they informed that the gender_submission is an example of the format we have to copie for the submission. Also, they don't makes available the ground_truth of the test file, only for the training. 

In [9]:
train_ds

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### C) Exploratory Analysis

### D) Summary